Загрузка модели FastText от DeepPavlov

In [ ]:
!wget -P /root/input/ -c "http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin"

--2022-12-23 09:49:26--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin [following]
--2022-12-23 09:49:26--  https://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.bin
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
fasttext = FastText()
fasttext.file_name='/root/input/ft_native_300_ru_twitter_nltk_word_tokenize.bin'
fasttext.load_binary_data()

Загрузка и предобработка обучающих данных

In [ ]:
from lxml import etree
import csv
import numpy as np
from typing import List, Tuple

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [ ]:
texts,labels=load_sentirueval_2016('bank_train_2016.xml')

In [ ]:
label_to_num={'negative':0,'neutral':1,'positive':2}

In [ ]:
num_labels=[label_to_num[label] for label in labels]

In [ ]:
from sklearn.model_selection import train_test_split
import random
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, num_labels, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
vocab=[]
for text in texts:
  tokenized_text=word_tokenize(text)
  for word in tokenized_text:
    if word not in vocab:
      vocab.append(word)

In [ ]:
embedding_matrix=np.zeros((len(vocab)+1, 100))
for i,word in enumerate(vocab):
  embedding_matrix[i,0:100]=fasttext[word]

<ipython-input-13-7649737db746>:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_matrix[i,0:100]=fasttext[word]


In [ ]:
max_length=max([len(word_tokenize(text)) for text in texts])

In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
encoded_docs_train = tokenizer.texts_to_sequences(train_texts)
X_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
y_train=np.asarray(train_labels)

In [ ]:
encoded_docs_val = tokenizer.texts_to_sequences(val_texts)
X_val = pad_sequences(encoded_docs_val, maxlen=max_length, padding='post')
y_val=np.asarray(val_labels)

In [ ]:
X_train.shape,y_train.shape

((8452, 46), (8452,))

In [ ]:
X_val.shape,y_val.shape

((940, 46), (940,))

In [ ]:
train_labels.count(0),train_labels.count(1), train_labels.count(2), len(y_train)

(1547, 6272, 633, 8452)

In [ ]:
X_train

array([[    7,    12,   147, ...,     0,     0,     0],
       [  436,     5,   230, ...,     0,     0,     0],
       [ 3112,   237,    38, ...,     0,     0,     0],
       ...,
       [ 4605,     4,     5, ...,     0,     0,     0],
       [  123,     5,     9, ...,     0,     0,     0],
       [16340,    40,  1109, ...,     0,     0,     0]], dtype=int32)

In [ ]:
X_val

array([[  139,    17,  5343, ...,     0,     0,     0],
       [  123,   205,   388, ...,     0,     0,     0],
       [ 2160,  3701,    90, ...,     0,     0,     0],
       ...,
       [   28,    52,    21, ...,     0,     0,     0],
       [    8,    15,     6, ...,     0,     0,     0],
       [15011,    92,   251, ...,     0,     0,     0]], dtype=int32)

Создание и обучение нейросети

In [ ]:
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
from tensorflow.keras.layers import Flatten, Conv1D, MaxPooling1D, SpatialDropout1D,Dense,Dropout,Embedding
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential
from keras.initializers import he_uniform, glorot_uniform
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [ ]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=2e-3,
    decay_steps=2 * len(X_train),
    decay_rate=0.9
)

In [ ]:
cnn = Sequential()
cnn.add(Embedding(len(vocab)+1, 100,weights=[embedding_matrix], input_length=max_length, trainable=False))
cnn.add(Conv1D(64, 2, padding='valid', activation='relu',
               kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_Block1_Layer1'))
cnn.add(Conv1D(64, 2, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_Block1_Layer2'))
cnn.add(MaxPooling1D(pool_size=2, name='MaxPool1'))
cnn.add(SpatialDropout1D(rate=0.15, name='SpatialDropout1', seed=RANDOM_SEED))

cnn.add(Conv1D(64, 3, padding='valid', activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_Block2_Layer1'))
cnn.add(Conv1D(64, 3, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_Block2_Layer2'))
cnn.add(MaxPooling1D(pool_size=2, name='MaxPool2'))
cnn.add(SpatialDropout1D(rate=0.15, name='SpatialDropout2', seed=RANDOM_SEED))

cnn.add(Flatten())
cnn.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED), name='HiddenLayer'))
cnn.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DropoutAfterHidden'))
cnn.add(Dense(3, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED), name='OutputLayer'))
cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 46, 100)           2165600   
                                                                 
 Conv_Block1_Layer1 (Conv1D)  (None, 45, 64)           12864     
                                                                 
 Conv_Block1_Layer2 (Conv1D)  (None, 44, 64)           8256      
                                                                 
 MaxPool1 (MaxPooling1D)     (None, 22, 64)            0         
                                                                 
 SpatialDropout1 (SpatialDro  (None, 22, 64)           0         
 pout1D)                                                         
                                                                 
 Conv_Block2_Layer1 (Conv1D)  (None, 20, 64)           12352     
                                                        

In [ ]:
BATCH_SIZE=128
cnn.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    shuffle=True, epochs=100,
    callbacks=[
        EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
        )
    ],
    verbose=1)

Epoch 1/100
67/67 [==============================] - 9s 107ms/step - loss: 0.6737 - sparse_categorical_accuracy: 0.7473 - val_loss: 0.5264 - val_sparse_categorical_accuracy: 0.7957
Epoch 2/100
67/67 [==============================] - 5s 71ms/step - loss: 0.5220 - sparse_categorical_accuracy: 0.7916 - val_loss: 0.4647 - val_sparse_categorical_accuracy: 0.8053
Epoch 3/100
67/67 [==============================] - 4s 53ms/step - loss: 0.4853 - sparse_categorical_accuracy: 0.8076 - val_loss: 0.4389 - val_sparse_categorical_accuracy: 0.8160
Epoch 4/100
67/67 [==============================] - 4s 58ms/step - loss: 0.4463 - sparse_categorical_accuracy: 0.8231 - val_loss: 0.4401 - val_sparse_categorical_accuracy: 0.8160
Epoch 5/100
67/67 [==============================] - 4s 58ms/step - loss: 0.4259 - sparse_categorical_accuracy: 0.8309 - val_loss: 0.4171 - val_sparse_categorical_accuracy: 0.8340
Epoch 6/100
67/67 [==============================] - 4s 55ms/step - loss: 0.3943 - sparse_categoric

Оценка качества классификации

In [ ]:
test_texts,test_labels=load_sentirueval_2016('banks_test_etalon.xml')

In [ ]:
test_num_labels=[label_to_num[label] for label in test_labels]

In [ ]:
encoded_docs_test = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
y_test=np.asarray(test_num_labels)

In [ ]:
X_test

array([[ 64,   4,  25, ...,   0,   0,   0],
       [ 11,   2,   1, ...,   0,   0,   0],
       [ 64,   4,  25, ...,   0,   0,   0],
       ...,
       [  8,   7,  12, ...,   0,   0,   0],
       [ 60,  57, 173, ...,   0,   0,   0],
       [ 60,  57,  89, ...,   0,   0,   0]], dtype=int32)

In [ ]:
y_pred = np.argmax(cnn.predict(X_test, batch_size=128), axis=1)

26/26 [==============================] - 1s 16ms/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, target_names=label_to_num.keys(), digits=4))

              precision    recall  f1-score   support

    negative     0.4674    0.4954    0.4810       767
     neutral     0.7505    0.8333    0.7898      2238
    positive     0.1333    0.0065    0.0124       308

    accuracy                         0.6782      3313
   macro avg     0.4504    0.4451    0.4277      3313
weighted avg     0.6276    0.6782    0.6460      3313



In [ ]:
from sklearn.metrics import f1_score

In [ ]:
macro=f1_score(test_num_labels,y_pred,average='macro',labels=[0,2])
micro=f1_score(test_num_labels,y_pred,average='micro',labels=[0,2])
macro,micro

(0.24669827957832033, 0.4014713610089332)

#Аугментация

In [ ]:
pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.2.0
    Uninstalling importlib-metadata-5.2.0:
      Successfully uninstalled importlib-metadata-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 5.2.0 which is incompatible.


In [ ]:
from textattack.transformations import WordSwapRandomCharacterDeletion
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import CompositeTransformation


from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Класс WordSwapQWERTY от textAttack не поддерживает кириллицу, поэтому я попыталась вручную решить эту проблему, изменив свойства keyboard_adjacency

In [ ]:
class RusWordSwapQWERTY(WordSwapQWERTY):
  def __init__(
        self, random_one=True, skip_first_char=False, skip_last_char=False, **kwargs
    ):
        super().__init__(**kwargs)
        self.random_one = random_one
        self.skip_first_char = skip_first_char
        self.skip_last_char = skip_last_char

        self._keyboard_adjacency = {
            "ё":["е"],
            "й": ["ц","ф","ё"],
            "ц": ["й", "у", "ф", "ы", "в"],
            "у": ["ц", "ы", "в", "а", "к"],
            "к": ["у", "в", "а", "п", "е"],
            "е": ["к", "а", "п", "р", "н"],
            "н": ["е", "п", "р", "о", "г"],
            "г": ["н", "р", "о", "л", "ш"],
            "ш": ["г", "о", "л", "д", "щ"],
            "щ": ["ш", "л", "д", "з"],
            "з": ["щ", "д","ж","э","х"],
            "х": ["з", "ж","э","ъ"],
            "ъ": ["х", "ж","э"],
            "ф": ["й", "ц", "ы", "я", "ч"],
            "ы": ["й", "ц", "у", "а", "в", "я", "ч"],
            "в": ["ц", "у", "к", "а", "с", "ч", "ы"],
            "а": ["у", "к", "е", "п", "м", "с", "в"],
            "п": ["к", "е", "н", "р", "и", "м", "в"],
            "р": ["е", "н", "г", "п", "о", "и", "т"],
            "о": ["н", "г", "ш", "л", "ь", "т", "р"],
            "л": ["г", "ш", "щ", "д", "ь", "о"],
            "д": ["ш", "щ", "з", "л","ж","ю","б"],
            "ж": ["д", "щ", "з", "х","э","ю"],
            "э": ["ж", "з", "х", "ъ","ю"],
            "я": ["ф", "ы", "ч"],
            "ч": ["ы", "в", "я", "с"],
            "с": ["ч", "в", "а", "м"],
            "м": ["с", "а", "п", "и"],
            "и": ["м", "п", "р", "т"],
            "т": ["и", "р", "о", "ь"],
            "ь": ["т", "о", "л","б"],
            "б": ["ь", "л", "д","ю"],
            "ю": ["б", "д", "ж"]
        }
  def _get_adjacent(self, s):
        s_lower = s.lower()
        if s_lower in self._keyboard_adjacency:
            adjacent_keys = self._keyboard_adjacency[s_lower]
            if s.isupper():
                return [key.upper() for key in adjacent_keys]
            else:
                return adjacent_keys
        else:
            return []

  def _get_replacement_words(self, word):
        if len(word) <= 1:
            return []

        candidate_words = []

        start_idx = 1 if self.skip_first_char else 0
        end_idx = len(word) - (1 + self.skip_last_char)

        if start_idx >= end_idx:
            return []

        if self.random_one:
            i = random.randrange(start_idx, end_idx + 1)
            candidate_word = (
                word[:i] + random.choice(self._get_adjacent(word[i])) + word[i + 1 :]
            )
            candidate_words.append(candidate_word)
        else:
            for i in range(start_idx, end_idx + 1):
                for swap_key in self._get_adjacent(word[i]):
                    candidate_word = word[:i] + swap_key + word[i + 1 :]
                    candidate_words.append(candidate_word)

        return candidate_words

In [ ]:
typo_transformation = CompositeTransformation([WordSwapRandomCharacterDeletion(),RusWordSwapQWERTY(random_one=False)])
typo_constraints = [RepeatModification(),StopwordModification(language='russian')]
typo_augmenter = Augmenter(transformation=typo_transformation, constraints=typo_constraints, pct_words_to_swap=0.5, transformations_per_example=5)

In [ ]:
s = texts[1]
typo_augmenter.augment(s)

['Мненик о креюитной карье чтб 24 http://t.co/SBJTcqjCg',
 'Мненип о кредитньй канте ытб 2 http://t.co/SBJTcsqjCg',
 'Мненип о кредптной катте стб 24 http://t.o/SBJTcsqjCg',
 'Мненме о кредттной карта втб 24 http://t.o/SBJTcsqjC',
 'Пнение о креюитной карте вьб 24 htp://t.co/SJTcsqjCg']

Аугментация обучающей выборки опечатками 

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
texts_with_typos=[]
labels_of_texts_with_typos=[]
for indx,text in enumerate(tqdm(train_texts)):
    texts_with_typos.extend(typo_augmenter.augment(text))
    labels_of_texts_with_typos.extend([train_labels[indx]]*5)

  0%|          | 0/8452 [00:00<?, ?it/s]

Запишем получившиеся тексты в csv-файл

In [ ]:
with open('texts_with_typos.csv', 'w', encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['text','label'])
    writer.writeheader()
    for i,text in enumerate(texts):
        writer.writerow({'text':text,'label':labels_of_texts_with_typos[i]})

Считывание полученных при аугментации текстов из файла

In [ ]:
with open('texts_with_typos.csv', 'r', encoding='utf-8') as file:
  texts_with_typos=[]
  labels_of_texts_with_typos=[]
  reader = csv.DictReader(file)
  for row in reader:
    if row!='':
      texts_with_typos.append(row['text'])
      labels_of_texts_with_typos.append(int(row['label']))

In [ ]:
print(len(labels_of_texts_with_typos),len(texts_with_typos))

9392 9392


In [ ]:
print(len(train_texts),len(train_labels))

8452 8452


In [ ]:
pos_texts_with_typos=[]
pos_labels_of_texts_with_typos=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]==2:
    pos_texts_with_typos.append(text)
    pos_labels_of_texts_with_typos.append(labels_of_texts_with_typos[text_idx])

In [ ]:
neg_texts_with_typos=[]
neg_labels_of_texts_with_typos=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]==0:
    neg_texts_with_typos.append(text)
    neg_labels_of_texts_with_typos.append(labels_of_texts_with_typos[text_idx])

In [ ]:
pos_neg_texts_with_typos=[]
pos_neg_labels_of_texts_with_typos=[]
for text_idx,text in enumerate(texts_with_typos):
  if labels_of_texts_with_typos[text_idx]!=1:
    pos_neg_texts_with_typos.append(text)
    pos_neg_labels_of_texts_with_typos.append(labels_of_texts_with_typos[text_idx])

#Аугментация с помощью BERT

In [ ]:
from textattack.transformations.word_swaps.word_swap_masked_lm import WordSwapMaskedLM

In [ ]:
bert_transformation = WordSwapMaskedLM(masked_language_model='DeepPavlov/rubert-base-cased')
bert_augmenter = Augmenter(transformation=bert_transformation, pct_words_to_swap=0.8)

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from time import time

In [ ]:
before=time()
print(bert_augmenter.augment(train_texts[0]))
time_passed=time()-before

['USD 24 проценты по кредитам [[UNK]]://time.com/b']


In [ ]:
time_passed/60

3.5640888094902037

In [ ]:
time_passed*len(train_texts)/(60*60*24)

20.91922126236889

Чтобы увеличить количество обучающих текстов в два раза понадобится почти 21 день. Поэтому этот вариант не подходит

In [ ]:
bert_tiny_transformation = WordSwapMaskedLM(masked_language_model='cointegrated/rubert-tiny')
bert_tiny_augmenter = Augmenter(transformation=bert_tiny_transformation, pct_words_to_swap=0.8)

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
before=time()
print(bert_tiny_augmenter.augment(text))
time_passed=time()-before

['Новая машина в кредит С Россельхозбанком просто и удобно']


In [ ]:
time_passed*len(train_texts)/(60*60*24)

0.2744440667386408

#Аугментация синонимами

In [ ]:
pip install spacy_udpipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 304 kB 8.3 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp38-cp38-linux_x86_64.whl size=5626932 sha256=899bb98377e3a32eccfa9202d6e879a2569eccb1e6fd8f617efe50018c8ffadf
  Stored in directory: /root/.cache/pip/wheels/d4/c1/67/142cea91540458ab9edac9c280a19b549a03217d7b441d32a6
Successfully built ufal.udpipe


In [ ]:
import spacy_udpipe

spacy_udpipe.download("ru") 
nlp = spacy_udpipe.load("ru")

Downloaded pre-trained UDPipe model for 'ru' language


In [ ]:
pip install wiki_ru_wordnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.6 MB 84.0 MB/s 
  Created wheel for wiki-ru-wordnet: filename=wiki_ru_wordnet-1.0.3-py3-none-any.whl size=20890442 sha256=7611bfe516d402fa2f3feef0623a1b63da7df0d6d08781f31a800eddbf79ae65
  Stored in directory: /root/.cache/pip/wheels/ff/06/93/b557ecda09fe9445d2d0543ee265b74ffe36c4835ae4f2dc41
Successfully built wiki-ru-wordnet


In [ ]:
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
rus_stopwords=stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def augment_with_synonym(text,n:int)->List[str]:
  doc=nlp(text.lower())
  lemmatized_text=[token.lemma_ for token in doc]
  changed_texts=[]
  for i in range(n):
    changed_text=[]
    c=1
    word_to_change=random.choice(lemmatized_text)
    changed_word=word_to_change
    while True:
      if c>len(lemmatized_text):
        break
      elif word_to_change not in rus_stopwords:
        synsets=wikiwordnet.get_synsets(word_to_change)
        if synsets:
         synset=random.choice(synsets)
         words=synset.get_words()
         new_word=random.choice(list(words))
         if new_word.lemma()!=word_to_change:
             changed_word=new_word.lemma()
         break
      word_to_change=random.choice(lemmatized_text)
      changed_word=word_to_change
      c+=1
    for word in lemmatized_text:
      if word == word_to_change:
        changed_text.append(changed_word)
      else:
        changed_text.append(word)
    if changed_text!=lemmatized_text:
      changed_texts.append(' '.join(changed_text))
  return set(changed_texts)

In [ ]:
def augment_with_synonyms(text,n:int)->List[str]:
  doc=nlp(text.lower())
  lemmatized_text=[token.lemma_ for token in doc]
  changed_texts=[]
  for i in range(n):
    changed_text=[]
    for word in lemmatized_text:
      if word not in rus_stopwords:
        synsets = wikiwordnet.get_synsets(word)
        if synsets:
          synset=random.choice(synsets)
          words=synset.get_words()
          new_word=random.choice(list(words))
          if new_word.lemma()!=word:
            changed_text.append(new_word.lemma())
        else:
         changed_text.append(word)
      else:
        changed_text.append(word)
    if changed_text!=lemmatized_text:
      changed_texts.append(' '.join(changed_text))
  return set(changed_texts)

In [ ]:
texts_with_synonyms=[]
labels_of_texts_with_synonyms=[]
for text_idx,text in enumerate(tqdm(train_texts)):
  if train_labels[text_idx]==2:
    augmented_texts=augment_with_synonyms(text,5)
    texts_with_synonyms.extend(augmented_texts)
    labels_of_texts_with_synonyms.extend([train_labels[text_idx]]*len(augmented_texts))

  0%|          | 0/8452 [00:00<?, ?it/s]

In [ ]:
print(len(texts_with_synonyms),len(labels_of_texts_with_synonyms))

2035 2035


In [ ]:
texts_with_synonyms[:20]

[': # news',
 ': недавний # news',
 ': бросить редакция # news',
 ': свежий # news',
 'райффайзенбанк редакция r-mobile : в последний r-mobile исп . . . http://t.co/bqbxgjirt0',
 'райффайзенбанк кинуть свежий передвижной использование r-mobile : в свежий r-mobile исп . . . http://t.co/bqbxgjirt0',
 'райффайзенбанк свежий применение r-mobile : в r-mobile исп . . . http://t.co/bqbxgjirt0',
 'райффайзенбанк кинуть редакция сотовый r-mobile : в использование r-mobile исп . . . http://t.co/bqbxgjirt0',
 'райффайзенбанк использование r-mobile : в r-mobile исп . . . http://t.co/bqbxgjirt0',
 '1,5 потребительский в выдавать #r',
 '@alfa_bank синхронно ) пре . вот бы так в ажиотаж на братиславский',
 '@alfa_bank не уже ли единовременно ) пре . вот бы так в ажиотаж на братиславский',
 '@alfa_bank не уже ли неурезанный единовременно ) пре . вот бы так в ажиотаж на братиславский',
 '@alfa_bank ) пре фантастика . вот бы так в ажиотаж на братиславский',
 '@alfa_bank синхронно ) пре фантастика . вот 

#Обучение модели и оценка качества

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, num_labels, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
train_texts.extend(texts_with_synonyms)
train_labels.extend(labels_of_texts_with_synonyms)

In [ ]:
train_texts.extend(pos_neg_texts_with_typos)
train_labels.extend(pos_neg_labels_of_texts_with_typos)

In [ ]:
train_labels.count(0),train_labels.count(1), train_labels.count(2), len(y_train)

(3117, 6272, 3378, 8452)

In [ ]:
encoded_docs_train = tokenizer.texts_to_sequences(train_texts)
X_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
y_train=np.asarray(train_labels)

In [ ]:
cnn = Sequential()
cnn.add(Embedding(len(vocab)+1, 100,weights=[embedding_matrix], input_length=max_length, trainable=False))
cnn.add(Conv1D(64, 3, padding='valid', activation='relu',
               kernel_initializer=he_uniform(seed=RANDOM_SEED), name='Conv_Block1_Layer1'))
cnn.add(Conv1D(64, 3, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),
               name='Conv_Block1_Layer2'))
cnn.add(MaxPooling1D(pool_size=2, name='MaxPool1'))
cnn.add(SpatialDropout1D(rate=0.15, name='SpatialDropout1', seed=RANDOM_SEED))

cnn.add(Conv1D(64, 4, padding='valid', activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_Block2_Layer1'))
cnn.add(Conv1D(64, 4, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED),name='Conv_Block2_Layer2'))
cnn.add(MaxPooling1D(pool_size=2, name='MaxPool2'))
cnn.add(SpatialDropout1D(rate=0.15, name='SpatialDropout2', seed=RANDOM_SEED))

cnn.add(Flatten())
cnn.add(Dense(512, activation='relu', kernel_initializer=he_uniform(seed=RANDOM_SEED), name='HiddenLayer'))
cnn.add(Dropout(rate=0.5, seed=RANDOM_SEED, name='DropoutAfterHidden'))
cnn.add(Dense(3, activation='softmax', kernel_initializer=glorot_uniform(seed=RANDOM_SEED), name='OutputLayer'))
cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 46, 100)           2165600   
                                                                 
 Conv_Block1_Layer1 (Conv1D)  (None, 44, 64)           19264     
                                                                 
 Conv_Block1_Layer2 (Conv1D)  (None, 42, 64)           12352     
                                                                 
 MaxPool1 (MaxPooling1D)     (None, 21, 64)            0         
                                                                 
 SpatialDropout1 (SpatialDro  (None, 21, 64)           0         
 pout1D)                                                         
                                                                 
 Conv_Block2_Layer1 (Conv1D)  (None, 18, 64)           16448     
                                                      

In [ ]:
BATCH_SIZE=128
cnn.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), shuffle=True, epochs=100, callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)], verbose=1)

Epoch 1/100
100/100 [==============================] - 16s 122ms/step - loss: 0.9940 - sparse_categorical_accuracy: 0.5398 - val_loss: 0.6774 - val_sparse_categorical_accuracy: 0.7543
Epoch 2/100
100/100 [==============================] - 8s 75ms/step - loss: 0.8921 - sparse_categorical_accuracy: 0.6185 - val_loss: 0.6400 - val_sparse_categorical_accuracy: 0.7798
Epoch 3/100
100/100 [==============================] - 7s 66ms/step - loss: 0.8229 - sparse_categorical_accuracy: 0.6615 - val_loss: 0.6653 - val_sparse_categorical_accuracy: 0.7553
Epoch 4/100
100/100 [==============================] - 7s 66ms/step - loss: 0.7652 - sparse_categorical_accuracy: 0.6937 - val_loss: 0.6384 - val_sparse_categorical_accuracy: 0.7862
Epoch 5/100
100/100 [==============================] - 7s 73ms/step - loss: 0.7213 - sparse_categorical_accuracy: 0.7183 - val_loss: 0.6535 - val_sparse_categorical_accuracy: 0.7851
Epoch 6/100
100/100 [==============================] - 7s 74ms/step - loss: 0.6924 - spa

In [ ]:
y_pred = np.argmax(cnn.predict(X_test, batch_size=128), axis=-1)

26/26 [==============================] - 1s 16ms/step


Результат аугментации положительных текстов синонимами (с увеличением в 5 раз), при которой в тексте по возможности заменяется каждое слово (а не только одно) и аугментацией положительных и отрицательных текстов опечатками. Это лучший результат, которого удалось достичь.

In [ ]:
print(classification_report(y_test, y_pred, target_names=label_to_num.keys(), digits=4))

              precision    recall  f1-score   support

    negative     0.3163    0.3559    0.3350       767
     neutral     0.6965    0.5590    0.6202      2238
    positive     0.1269    0.2695    0.1726       308

    accuracy                         0.4851      3313
   macro avg     0.3799    0.3948    0.3759      3313
weighted avg     0.5556    0.4851    0.5126      3313



Качество распознавания нейтральных текстов заменто упало, поэтому общие метрики стали ниже, чем для исходных данных, но положительные и в меньшей степени отрицательные твиты стали классифицироваться чуть лучше.

In [ ]:
macro=f1_score(test_num_labels,y_pred,average='macro',labels=[0,2])
micro=f1_score(test_num_labels,y_pred,average='micro',labels=[0,2])
macro,micro

(0.2537632488552734, 0.27469135802469136)